# Skip the Docs with Mocks

 > #### COhPy - Jan 25, 2016

 > #### Andrew Kubera

## What are Mocks?

* Objects that allow any method to be called, or attribute to be retrieved.
  * You may set the return value
  * Default: all methods return a new mock
* Useful for writing tests
  * Simplify 'complex' objects & operations like sockets, HTTP Server Responses, database state

* ***Tracks all calls made to it***

## How to use Mock

In [1]:
from unittest import mock

In [2]:
m = mock.Mock()
m.foo()
m.mock_calls

[call.foo()]

In [3]:
m = mock.Mock()
m.spam(1, a=5)
m.mock_calls

[call.spam(1, a=5)]

In [4]:
m = mock.Mock()
m.spam(1)
m.spam(2)
m.spam(3)
m.mock_calls

[call.spam(1), call.spam(2), call.spam(3)]

In [5]:
m.spam(100).next()
m.spam(200).next()
m.mock_calls

[call.spam(1),
 call.spam(2),
 call.spam(3),
 call.spam(100),
 call.spam().next(),
 call.spam(200),
 call.spam().next()]

## Use to Investigate How Libraries Work
---
#### Why?

* Don't have the docs with you?
* Bad documentation?
* Fun?

In [6]:
def someones_function(obj):
    ...
    obj.open_connection('http://their.secret.url/data')
    ...
    return obj.read()    

In [7]:
m = mock.Mock()
someones_function(m)
m.mock_calls


[call.open_connection('http://their.secret.url/data'), call.read()]

### One more thing...

In [8]:
m = mock.Mock()
len(m)
m.mock_calls

TypeError: object of type 'Mock' has no len()

### Solution: MagicMock

In [9]:
m = mock.MagicMock()
len(m)
m.mock_calls

[call.__len__()]

* Most builtin/standard python libraries use *magics*

### Magics may return non-mock objects

In [10]:
m = mock.MagicMock()
print('len:', m.__len__())
print('int:', m.__int__())
print('float:', m.__float__())
print('item:', m[22])

len: 0
int: 1
float: 1.0
item: <MagicMock name='mock.__getitem__()' id='4437378608'>


In [11]:
m.__len__()

0

### But not your own magic...

In [12]:
m = mock.MagicMock()
m.__foo__()
m.mock_calls

AttributeError: __foo__

In [13]:
m = mock.MagicMock()
m._foo__()
m.mock_calls

[call._foo__()]

### Now lets play with mocks

In [14]:
m = mock.MagicMock()
"%f" % m
m.mock_calls

[call.__float__()]

In [15]:
m = mock.MagicMock()
"%d %s %E" % (m, m, m)
m.mock_calls

[call.__int__(), call.__str__(), call.__float__()]

In [16]:
class Foo:
    
    def __div__(self, v):
        pass


In [17]:
foo = Foo()
x = foo / 4

TypeError: unsupported operand type(s) for /: 'Foo' and 'int'

### Why Doesn't foo / 4 work??

In [18]:
m = mock.MagicMock()
m / 2
m.mock_calls

[call.__truediv__(2)]

In [19]:
m = mock.MagicMock()
m // 2
m.mock_calls

[call.__floordiv__(2)]

In [20]:
m = mock.MagicMock()
2 / m
m // 2
m % 2
m * 2
m ** 2
m.mock_calls

[call.__rtruediv__(2),
 call.__floordiv__(2),
 call.__mod__(2),
 call.__mul__(2),
 call.__pow__(2)]

In [21]:
m = mock.MagicMock()
m.__iadd__.return_value = m
print(id(m))
m + 2
m += 2
print(id(m))
m.mock_calls

4437662744
4437662744


[call.__add__(2), call.__iadd__(2)]

In [22]:
l, t, s = mock.MagicMock(), mock.MagicMock(), mock.MagicMock()
list(l)
tuple(t)
set(s);

In [23]:
l.mock_calls

[call.__iter__(), call.__len__()]

In [24]:
t.mock_calls

[call.__iter__(), call.__len__()]

In [25]:
s.mock_calls

[call.__iter__()]

## How can my class be 'range-able'?

In [26]:
m = mock.MagicMock()
range(m)
m.mock_calls

[call.__index__()]

In [27]:
m = mock.MagicMock()
[x for x in m]
m.mock_calls

[call.__iter__()]

In [28]:
import numpy as np
m = mock.MagicMock()
np.array(m)
m.mock_calls

[call.__len__(),
 call.__iter__(),
 call.__len__(),
 call.__iter__(),
 call.__len__(),
 call.__iter__()]

### Limitations

* Mocks do not overload `__getattr__` and `__repr__`
* `__repr__` is always used the same - no big deal
* Get around attributes
  * Store result of dir
  * compare with dir after your code 

In [29]:
import numpy as np
m = mock.MagicMock()
np.shape(m)
m.mock_calls

[]

## Default Mock Attributes/Properties Set

In [30]:
m = mock.MagicMock()
default_attrs = set(dir(m))
default_attrs

{'assert_any_call',
 'assert_called_once_with',
 'assert_called_with',
 'assert_has_calls',
 'assert_not_called',
 'attach_mock',
 'call_args',
 'call_args_list',
 'call_count',
 'called',
 'configure_mock',
 'method_calls',
 'mock_add_spec',
 'mock_calls',
 'reset_mock',
 'return_value',
 'side_effect'}

In [31]:
np.shape(m)
set(dir(m)) - default_attrs

{'shape'}

# End.